# Ticket SITCOM-818 - supporting technote SITCOMTN-083
Craig Lage - 15-May-23 \
Updated 16-Oct-23 with new FATable syntax \
Updated 14-Nov-23 with better plotting\
Updated 05-Sep-24: \
     Force it to use the most recent bump test\
     try-except loops to force it to plot what it has when there are fails.

Here is what was requested:
A script is needed to return maximal overshoot values per M1M3 during bump testing. It shall retrieve FA following error (M1M3.forceActuatorData.primaryCylinderFollowingError and M1M3.forceActuatorData.secondaryCylinderFollowingError) and per actuator, while it is bump tested, retrieve min and max (absolute) value of the deviation.

This notebook does those things


## Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)

start = "2024-10-12T00:00:00"
end = "2024-10-12T23:30:00"

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
async def plot_bump_test_following_errors(fig, bumps, id, index=0):
    """ Plot a visualization of the bump test following errors
        Parameters
        ----------
        fig : a matplotlib figure object

        bumps: pandas dataframe
            This is a dataframe containg the bump test status
        
        id: 'int'
            The actuator id desired

        Returns
        -------
        No return, only the fig object which was input
    """
    
    this_bump = bumps[bumps['actuatorId']==id]
    last_this_bump_index = bumps[bumps['actuatorId']==id].last_valid_index()
    fa = force_actuator_from_id(id)
    primary_bump = f"primaryTest{fa.index}"
    primary_follow = f"primaryCylinderFollowingError{fa.index}"
    primary_force = f"primaryCylinderForce{fa.index}"
    if fa.actuator_type.name == 'DAA':
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_force = f"secondaryCylinderForce{fa.s_index}"
        secondary_follow = f"secondaryCylinderFollowingError{fa.s_index}"
        secondary_name = fa.orientation.name
    else:
        secondary_name = None
        secondary_force = None
        secondary_follow = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}", fontsize=18)
    index = len(this_bump[this_bump[primary_bump]==2]['timestamp'].values) - 1 # Try using the most recent one
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[index] - 1.0
    plot_end = plot_start + 14.0 
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, primary_follow, 'timestamp'], start.utc, end.utc)
    timestamp = forces.index[0].isoformat().split('.')[0]
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}\n {timestamp}", fontsize=18)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    primary_errors = forces[primary_follow].values
    plot_start -= t0
    plot_end -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, primary_forces, label='Data')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (N)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.title("Following Errors - Log scale above 10 N")
    plt.plot(times, primary_errors)
    plt.ylim(-100,100)
    plt.yscale('symlog', linthresh=10)
    plt.plot([plot_start, plot_end], [5.0,5.0], ls='--', color='red')
    plt.plot([plot_start, plot_end], [-5.0,-5.0], ls='--', color='red')
    plt.plot([plot_start, plot_end], [2.5,2.5], ls='--', color='green')
    plt.plot([plot_start, plot_end], [-2.5,-2.5], ls='--', color='green')
    time_pass_primary_pos = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - t0 + 3.0
    plt.plot([time_pass_primary_pos, time_pass_primary_pos], [-100,100], ls='--', color='black')
    try:
        time_pass_primary_neg = this_bump[this_bump[primary_bump]==4]['timestamp'].values[0] - t0 + 3.0
    except:
        time_pass_primary_neg = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - t0 + 9.0
    plt.plot([time_pass_primary_neg, time_pass_primary_neg], [-100,100], ls='--', color='black')
    try:
        pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
        if pass_fail[primary_bump] == 6:
            plt.text(1.0, 60.0, "PASSED", color='g')
        elif pass_fail[primary_bump] == 7:
            plt.text(1.0, 60.0, "FAILED", color='r')
            print(f"Primary {id} failed the bump test")
    except:
        plt.text(1.0, 60.0, "FAILED", color='r')
        print(f"Primary {id} failed the bump test")
        

    plt.text(9, 65, f"Max = {np.max(primary_errors):.1f} N")
    plt.text(5, -80, f"Min = {np.min(primary_errors):.1f} N")
    plt.xlim(plot_start, plot_end)
    plt.yticks([-100,-10,-7.5,-5.0,-2.5,0,2.5,5.0,7.5,10.0,100])
    plt.xlabel("Time (seconds)")
    plt.ylabel("Following Errors (N)")
    try:
        if secondary_name is not None:
            plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[index] - 1.0
            plot_end = plot_start + 14.0
            start = Time(plot_start, format='unix_tai', scale='tai')
            end = Time(plot_end, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                     [secondary_force, secondary_follow, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            secondary_forces = forces[secondary_force].values
            secondary_errors = forces[secondary_follow].values
            plot_start -= t0
            plot_end -= t0
            plt.subplot(2,2,2)
            plt.title(f"Secondary - {secondary_name}")
            plt.plot(times, secondary_forces, label='Data')
            plt.xlim(plot_start, plot_end)
            plt.ylim(-400,400)
            plt.xlabel("Time (seconds)")
            plt.ylabel("Force (N)")
            plt.legend()
            plt.subplot(2,2,4)
            plt.title("Following Errors - Log scale above 10 N")
            plt.plot(times, secondary_errors)
            plt.yscale('symlog', linthresh=10)
            plt.plot([plot_start, plot_end], [5.0,5.0], ls='--', color='red')
            plt.plot([plot_start, plot_end], [-5.0,-5.0], ls='--', color='red')
            plt.plot([plot_start, plot_end], [2.5,2.5], ls='--', color='green')
            plt.plot([plot_start, plot_end], [-2.5,-2.5], ls='--', color='green')
            try:
                time_pass_secondary_pos = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - t0 + 3.0
                plt.plot([time_pass_secondary_pos, time_pass_secondary_pos], [-100,100], ls='--', color='black')
                time_pass_secondary_neg = this_bump[this_bump[secondary_bump]==4]['timestamp'].values[0] - t0 + 3.0
                plt.plot([time_pass_secondary_neg, time_pass_secondary_neg], [-100,100], ls='--', color='black')
            except:
                pass
            try:
                pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
                if pass_fail[secondary_bump] == 6:
                    plt.text(10.2, 60.0, "PASSED", color='g')
                elif pass_fail[secondary_bump] == 7:
                    plt.text(1.0, 60.0, "FAILED", color='r')
                    print(f"Secondary {id} failed the bump test")
            except:
                plt.text(1.0, 60.0, "FAILED", color='r')
                print(f"Secondary {id} failed the bump test")
        
            plt.text(9, 65, f"Max = {np.max(secondary_errors):.1f} N")
            plt.text(5, -80, f"Min = {np.min(secondary_errors):.1f} N")
            plt.xlim(plot_start, plot_end)
            plt.ylim(-100, 100)
            plt.yticks([-100,-10,-7.5,-5.0,-2.5,0,2.5,5.0,7.5,10.0,100])
            plt.xlabel("Time (seconds)")
            plt.ylabel("Following Errors (N)")
        else:
            plt.subplot(2,2,2)
            plt.title("No Secondary")
            plt.xticks([])
            plt.yticks([])
            plt.subplot(2,2,4)
            plt.xticks([])
            plt.yticks([])
    except:
        pass
    return 

## First run just one actuator

In [ ]:
client = EfdClient('usdf_efd')
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))
len(bumps)

In [ ]:
id = 131
fa = force_actuator_from_id(id)
print(fa.index, fa.s_index)


In [ ]:
%matplotlib inline
client = EfdClient('usdf_efd')
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))
# The actuator id runs from 101 to 443, as described in 
# Section 2 of https://sitcomtn-083.lsst.io/
id = 131
fig = plt.figure(figsize=(10,10))
await plot_bump_test_following_errors(fig, bumps, id, index=5)
plt.savefig(str(data_dir / f"Bump_Test_Following_Errors_{id}.png"))

# Now run the whole bump test

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(str(data_dir / f"Bump_Test_Following_Errors_{timestamp}.pdf"))
fig = plt.figure(figsize=(10,10))
for index in range(len(FATable)):
    try:
        id = FATable[index].actuator_id
        await plot_bump_test_following_errors(fig, bumps, id, index=0)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        print(f"Plot for actuator {id} succeeded!")
        plt.clf()
    except:
        print(f"Plot for actuator {id} failed!")
        continue
pdf.close()


# The cell below runs one actuator of a period of time

In [ ]:
id = 409
client = EfdClient('usdf_efd')
pdf = PdfPages(str(data_dir / f"Bump_Test_Following_Errors_{id}.pdf"))
fig = plt.figure(figsize=(10,10))

startDay = 20240901
endDay = 20240913

dayObs = startDay
while dayObs < endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    
    start = f"{dayString}T12:00:00"
    end = f"{nextDayString}T12:00:00"

    try:
        bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                                Time(start, scale='utc'), Time(end, scale='utc'))
        this_bump = bumps[bumps['actuatorId']==id]
        fa = force_actuator_from_id(id)
        primary_bump = f"primaryTest{fa.index}"
        start_times = this_bump[this_bump[primary_bump]==2]['timestamp'].values
        print("Num start times", len(start_times))
        for index in range(len(start_times)):
            await plot_bump_test_following_errors(fig, bumps, id, index=index)
            pdf.savefig(fig)  # saves the current figure into a pdf page
            print(f"Plot for {dayObs} {index} succeeded!")
            plt.clf()
        dayObs = calcNextDay(dayObs)
    except:
        print(f"Plot for {dayObs} failed!")
        dayObs = calcNextDay(dayObs)
        continue
pdf.close()
